> ## 0. Import some required libraray

In [1]:
import string
import traceback
import re
import os
import json
import pickle
from PIL import Image
from time import time

import numpy as np
from numpy import array
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from tensorflow import keras
import tensorflow as tf

> ## 2. Load the Instrcuction File

In [2]:
def read_input(file_name):
    with open(file_name, 'r') as f:
        return f.readlines()

> ## 3. Crate A class for Parking System

In [3]:
class car:
    
    # This will create a parking system with given no of slot
    def __init__(self, no_of_slots):
        self.__data = pd.DataFrame(np.arange(1, no_of_slots+1), columns=['Slot no.'])
        self.__data['Registration no.'] = np.nan
        self.__data['Color'] = np.nan
        self.__data['Status'] = "No car Parked"
        self.__size = no_of_slots
        self.__free_slot = list(range(1, no_of_slots+1))
        print(f"Created a Parking lot with {no_of_slots} slots")
        
    # This method will allocate a slot which is free, If there is no free slot it give msg(Sorry Parking lot is full!)
    def allocate_slot(self, registration_no, color="black"):
        if len(self.__free_slot) != 0:
            status='Parked'
            free_slot = self.__free_slot.pop(0)
            mask = self.__data['Slot no.'] == free_slot
            self.__data.loc[mask, ['Registration no.', 'Color', 'Status']] = registration_no, color, status
            return f"Allocated Slot no. {free_slot}"
        else:
            return "Sorry Parking lot is full!"
        
    # This method will free the space occupied by the car
    def leave(self, slot_no):
        mask = self.__data['Slot no.'] == slot_no
        self.__data.loc[mask, ['Registration no.', 'Color', 'Status']] = np.nan, np.nan, "No Car Parked"
        self.__free_slot.append(slot_no)
        self.__free_slot.sort()
        return f"Slot no. {slot_no} is now free!"
    
    # This method will give details about parking on the basis of requirements
    def details(self, registration_no=None, color=None, status=None):
        
        # If registraion no is given
        if registration_no is not None:
            
            # If registraion no and color are given
            if color is not None:
                
                # If registraion no, color and status are given
                if status is not None:
                    temp = self.__data[self.__data['Registration no.'] == registration_no]
                    temp = temp[temp['Color'] == color]
                    temp = temp[temp['Status'] == status]
                    if temp.shape[0] != 0:
                        return temp
                    return 'Sorry, No data found!'
                
                # If registraion no and color are given but staus is not given
                else:
                    temp = self.__data[self.__data['Registration no.'] == registration_no]
                    temp = temp[temp['Color'] == color]
                    if temp.shape[0] != 0:
                        return temp
                    return 'Sorry, No data found!'
            
            # If registraion no is given but color is not given
            else: 
                
                # If registraion no and ststus are given but color is not given
                if status is not None:
                    temp = self.__data[self.__data['Registration no.'] == registration_no]
                    temp = temp[temp['Status'] == status]
                    if temp.shape[0] != 0:
                        return temp
                    return 'Sorry, No data found!'
                
                # If registraion no is given but status and color are not given
                else:
                    temp = self.__data[self.__data['Registration no.'] == registration_no]
                    if temp.shape[0] != 0:
                        return temp
                    return 'Sorry, No data found!'
                
        # If registration no is not given
        if registration_no is None and color is not None:
            
            # If status and color are given
            if status is not None:
                temp = self.__data[self.__data['Color'] == color]
                temp = temp[temp['Status'] == status]
                if temp.shape[0] != 0:
                    return temp
                return 'Sorry, No data found!'
            
            # If Only color is given
            else : 
                temp = self.__data[self.__data['Color'] == color]
                if temp.shape[0] != 0:
                    return temp
                return 'Sorry, No data found!'
            
        # If only status is given
        if registration_no is None and color is None and status is not None:
            temp = self.__data[self.__data['Status'] == status]
            if temp.shape[0] != 0:
                return temp
            return 'Sorry, No data found!'
        
        # If no arguments are provided
        else:
            return '''Please provide parameter to the details value
            You have to provide atleast one of the following parameter
                1. registration_no
                2. color
                3. status
            '''
    
    # This method will display the present situation of parking slot
    def status(self):
        return self.__data
            
    def __str__(self):
        return str(self.__data)

> ## 4. Create a Function Which Execute the given Instruction

In [4]:
def execute_instructions(instructions):
    print("\t"*5, "PARKING SYSTEM\n", "\t"*4, "-"*30, "\n")
    for instruction in instructions:
        print(instruction)
        instr = instruction.split(":")[1].lower().strip()
        instr_list = instr.split()
        
        # It will create a parking system with given no of slot
        if re.search("create.park.*lot.*\d", instr, flags=0) is not None:
            a = int(re.findall("\d{1,10}", instr)[0])
            c = car(a)
            
        # It will allocate a slot which is free, If there is no free slot it give msg(Sorry Parking lot is full!)
        elif 2 <= len(instr_list) <= 3 and instr_list[0] == "park":
            registration_no = instr_list[1]
            if len(instr_list) == 3:
                color = instr_list[2]
                print(c.allocate_slot(registration_no, color))
            else : print(c.allocate_slot(registration_no))
                
        # It will free the space occupied by the car
        elif re.search("leave.*\d", instr, flags=0) is not None:
            a = int(re.findall("\d{1,10}", instr)[0])
            print(c.leave(a))
            
        # It will display the present situation of parking slot
        elif instr == "status":
            print(c.status())
        elif re.search("details", instr, flags=0) is not None:
            if re.search("with", instr, flags=0) is not None:
                a = instr.split("with")[1]
                if re.search("colour", a, flags=0) is not None:
                    if re.search("registration.no", a, flags=0) is not None:
                        color = a.split("colour")[1].split()[0]
                        registration_no = re.split("registration.no", a)[1].strip()
                        print(c.details(color=color, registration_no = registration_no))
                    else:
                        color = a.split("colour")[1].split()[0]
                        print(c.details(color=color))
                elif re.search("registration.no", a, flags=0) is not None:
                    registration_no = re.split("registration.no", a)[1].strip()
                    print(c.details(registration_no = registration_no))
            elif re.search("which", instr, flags=0) is not None:
                status = re.split("status.is", instr)[1].strip()
                if status == "parked":
                    print(c.details(status="Parked"))
                else:
                    print(c.details(status = "No Car Parked"))
        else : 
            print("Please enter a valid Instruction!")
        print("*"*100, "\n")
    print("\t"*4, "-"*30, "\n", "\t"*5, "INSTRUCTION END\n", "\t"*4, "-"*30)

> ## 5. Create a main funnction
> * Load the instruction file
> * Execute the instruction file

In [5]:
if __name__ == "__main__":
    instructions = read_input('car_parking_input.txt')
    execute_instructions(instructions)

					 PARKING SYSTEM
 				 ------------------------------ 

instruction 01 : create parking lot 6 

Created a Parking lot with 6 slots
**************************************************************************************************** 

instruction 02 : park KA-01-HH-1234 White 

Allocated Slot no. 1
**************************************************************************************************** 

instruction 03 : park KA-01-HH-9999 White

Allocated Slot no. 2
**************************************************************************************************** 

instruction 04 : park KA-01-BB-0001 Black

Allocated Slot no. 3
**************************************************************************************************** 

instruction 05 : park KA-01-HH-7777 Red.

Allocated Slot no. 4
**************************************************************************************************** 

instruction 06 : park KA-01-HH-2701 Blue

Allocated Slot no. 5
*******************************